In [ ]:
import os
import numpy as np
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import random
from scipy import ndimage, misc
from sklearn.metrics import confusion_matrix
import glob

np.random.seed(8)

In [ ]:
def MyVGG16Model():
    # VGG16 default input
    input_tensor = Input(shape=(224, 224, 3))

    # load pretrained VGG16 network (only conv layer)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    print(vgg16.summary())

    # add dense layer to perform classification for 5 classes
    class_model = Sequential()
    class_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
    class_model.add(Dense(512, activation='relu'))
    class_model.add(Dropout(0.5))
    class_model.add(Dense(256, activation='relu'))
    class_model.add(Dropout(0.5))
    class_model.add(Dense(5, activation='softmax'))
    print(class_model.summary())

    # use transfer learning -> use trained feature for first layers and fine tuned the last one (conv+dense)
    model = Model(input=vgg16.input, output=class_model(vgg16.output))
    for layer in model.layers[:-5]:
        layer.trainable = False

    # model summary
    print(model.summary())
    for layer in model.layers:
        print(layer, layer.trainable)

    return model

In [ ]:
def addSampleFlipped(dir_path):
    """
    Increase the number of sample applying image transformation (rotation+flip)
    :param dir_path:
    :return:
    """
    for f in glob.glob(dir_path + "/*.png", recursive=True):
        img = misc.imread(f)

        img = np.fliplr(img)

        sign = -1
        if random.random() > 0.5:
            sign = 1

        img = ndimage.rotate(img, random.randint(2,20)*sign, mode='constant', cval=255)

        f_rot = os.path.splitext(f)[0] + "_rot" + ".png"
        misc.imsave(f_rot, img)
        

In [ ]:
def balanceDataset(path, path_val, min_file_per_class=500):
    """
    Split the dataset into training and validation with the same number of samples foreach training class
    :param path:
    :param path_val:
    :param min_file_per_class:
    :return:
    """
    data = []
    for i, dir in enumerate(os.listdir(path)):
        # validation directory tree
        if not os.path.exists(path_val + "/" + dir):
            os.makedirs(path_val + "/" + dir)

        # images files
        files = glob.glob(path + "/" + dir + "/*.png", recursive=False)
        num_imgs = len(files)

        # dataset already splitted
        if(num_imgs == min_file_per_class):
            break

        # add new samples for classes with few elements
        if num_imgs < min_file_per_class:
            addSampleFlipped(path + "/" + dir)
            files = glob.glob(path + "/" + dir + "/*.png", recursive=False)
            num_imgs = len(files)

        # randomly select validation file
        idx = random.sample(range(num_imgs), num_imgs - min_file_per_class)
        for i in idx:
            os.rename(files[i], path_val + files[i][files[i].find(path) + len(path):])
            

In [ ]:
def loadSamples(data, channels=3, img_rows=224, img_cols=224):
    """
    load images and label from file
    :param data: list of tuple (y, file_path)
    :param channels:
    :param img_rows:
    :param img_cols:
    :return:
    """
    # initialize output tensor
    X = np.zeros((len(data), img_rows, img_cols, channels))
    Y = np.zeros((len(data), 1))

    for i,(y, f) in enumerate(data):
        img = misc.imread(f)
        img = img.astype('float32')
        img = misc.imresize(img, (img_rows, img_cols, channels))
        img = img / 255

        Y[i] = y
        X[i] = img

    return X, Y


In [ ]:
def getSamplesData(path):
    """
    Load a list of tuple (label, file_path). Label is the numeric index of the directory
    :param path: 
    :return: 
    """
    data = []
    for i, dir in enumerate(os.listdir(path)):
        for f in glob.glob(path + "/" + dir + "/*.png", recursive=True):
            data.append([i, f])
    return data


In [ ]:
path_training = 'dati_maniche'
path_validation = 'dati_maniche_val'

# Create a balanced dataset: classes will have the same number of samples in training dataset (500)
# Training and validation are splitted randomly in 2 folder
balanceDataset(path_training, path_validation)

# Get model
model = MyVGG16Model()

#optimizer = SGD(lr=1e-4, decay=0.005, momentum=0.9, nesterov=True)
optimizer = RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# set training callbacks: stop if validation loss increase (probably is overfitting) and save weights with the lower loss
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
ckpt = ModelCheckpoint('weights/weights.{epoch:03d}-{val_loss:.5f}.hdf5', save_best_only=True, monitor='val_loss', mode='min')   

In [ ]:
# Use keras API to apply data augmentation (add random samples to our dataset to increase generalization)
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=False,
                         width_shift_range=10,
                         height_shift_range=0.2,
                         channel_shift_range=0,
                         zoom_range=0.2,
                         rescale=1. / 255,
                         rotation_range=10)
# On validation dataset apply only rescale
validation_datagen = ImageDataGenerator(rescale=1. / 255)

batch_size = 32

# Use keras API for loading the sample for training -> samples are automatically labeled from directory
# this API ensure images and label will be loaded in loop
train_generator = gen.flow_from_directory(
    path_training,
    target_size=(224, 224),     # Resize images
    color_mode='rgb',
    classes=None,
    class_mode='categorical',   # convert labels to categorical
    batch_size=batch_size,
    shuffle=True)

# Use keras API for loading the sample for training
validation_generator = validation_datagen.flow_from_directory(
    path_validation,
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False)



In [ ]:
# Train model with generator
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=50,
    callbacks=[earlyStopping, ckpt],
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size)





In [ ]:
# Verify on validation data -> it could be a better idea to use other examples but the number of samples was not so high
data = getSamplesData(path_validation)
X, Y = loadSamples(data)
Y = Y.reshape((Y.shape[0]))
print(Y)

print("Loading Weights...")
model.load_weights('weights/weights.hdf5')

pred = model.predict(X)
predicted_classes = np.argmax(pred, axis=1)  # take the argmax -> output with the higher score define the class

print(predicted_classes)

# numbers of miss prediction
errors = np.where(predicted_classes != Y)[0]
print("No of errors = {}/{}".format(len(errors), validation_generator.samples))

# confusion matrix
conf_matrix = confusion_matrix(Y, predicted_classes)
print(conf_matrix)

# in my test, there are a lot of bad prediction between maniche_a_34 and maniche_lunghe